<a href="https://colab.research.google.com/github/randaama/DS4002_Group2_Project2/blob/main/SCRIPTS/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sooo we need to look at the trends of the top  10 Counties
Orange County  
Miami Dade County
Hennepin County
Hillsborough County
Broward County
Maricopa County
Hamilton County
Pinellas County
Natrona County
Fullton county
to give a desirability determination based on precipitation and average temperature. Use that to build a desiribity() function 1. Predict the average temperature and precipitation of each month based on the change in the past and then stick that into the desirability function and then rank the top Counties.

In [214]:
import pandas as pd
import plotly.express as px
#import pyearth
clim = pd.read_csv('/Users/selahmitchell/climate_data_all.csv')

In [215]:
import plotly.express as px
import numpy as np

In [216]:
clim.head()

,year,fips,month,temp,County_Name,State,percip
0,2004,1001,Jan,45.0,Autauga,Alabama,2.60
1,2005,1001,Jan,49.9,Autauga,Alabama,3.08
2,2006,1001,Jan,52.0,Autauga,Alabama,5.69
3,2007,1001,Jan,47.3,Autauga,Alabama,4.73
4,2008,1001,Jan,43.4,Autauga,Alabama,4.36


In [217]:
data = [
    {"Rank": 1, "County_Name": "Orange", "State": "Florida"},  # Adjusted to match the format
    {"Rank": 2, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 3, "County_Name": "Hennepin", "State": "Minnesota"},
    {"Rank": 4, "County_Name": "Hillsborough", "State": "Florida"},
    {"Rank": 5, "County_Name": "Broward", "State": "Florida"},
    {"Rank": 6, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 7, "County_Name": "Hamilton", "State": "Ohio"},
    {"Rank": 8, "County_Name": "Pinellas", "State": "Florida"},
    {"Rank": 9, "County_Name": "Natrona", "State": "Wyoming"},
    {"Rank": 10, "County_Name": "Fulton", "State": "Georgia"},
]

counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_wallet= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_wallet)


      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     6
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     6
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     6
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     6
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     6
...    ...    ...   ...   ...         ...      ...     ...   ...
2395  2019  56025   Dec  25.0     Natrona  Wyoming    0.51     9
2396  2020  56025   Dec  25.5     Natrona  Wyoming    0.62     9
2397  2021  56025   Dec  27.7     Natrona  Wyoming    0.41     9
2398  2022  56025   Dec  21.0     Natrona  Wyoming    0.66     9
2399  2023  56025   Dec  30.3     Natrona  Wyoming    0.14     9

[2400 rows x 8 columns]


In [218]:
import pandas as pd

top10_wallet['year_month'] = top10_wallet['year'].astype(str) + '-' + top10_wallet['month']

# Calculate overall averages for each month across all years
overall_avg = top10_wallet.groupby(['month', 'year']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
overall_avg.columns = ['month', 'year', 'overall_avg_temp', 'overall_avg_percip']

# Merge overall averages with the monthly data
monthly_averages = pd.merge(top10_wallet, overall_avg, on=['month', 'year'])

    # Calculate absolute differences from overall averages
monthly_averages['temp_diff'] = abs(monthly_averages['temp'] - monthly_averages['overall_avg_temp'])
monthly_averages['percip_diff'] = abs(monthly_averages['percip'] - monthly_averages['overall_avg_percip'])

# Weights for desirability components
w_T = 0.3  # weight for temperature
w_P = 0.2  # weight for precipitation
w_r = 0.5  # weight for rank

# Normalize the rank so that a lower rank yields a higher desirability contribution
monthly_averages['rank_adjusted'] = monthly_averages['Rank'].max() - monthly_averages['Rank'] + 1

# Define maximum possible differences for normalization
max_temp_diff = monthly_averages['temp_diff'].max()
max_percip_diff = monthly_averages['percip_diff'].max()

# Calculate desirability score
monthly_averages['desirability_month'] = (
    (1 - (monthly_averages['temp_diff'] / max_temp_diff)) * w_T +  # Smaller temp_diff is better
    (1 - (monthly_averages['percip_diff'] / max_percip_diff)) * w_P +  # Smaller percip_diff is better
    (1 - (monthly_averages['rank_adjusted'] / monthly_averages['rank_adjusted'].max())) * w_r  # Lower rank is better
)

# Calculate yearly desirability score by taking the mean of monthly scores per county and year
yearly_desirability = monthly_averages.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']

# Merge yearly desirability back into the monthly averages DataFrame
monthly_averages = pd.merge(monthly_averages, yearly_desirability, on=['County_Name', 'State', 'year'], how='left')

# Output the final results
final_results = monthly_averages[['County_Name', 'State', 'temp', 'percip', 'desirability_month', 'Rank', 'month', 'year', 'desirability_year']]
print(final_results)

       County_Name      State  temp  percip  desirability_month  Rank month  \
0         Maricopa    Arizona  53.9    0.52            0.660293     6   Jan   
1          Broward    Florida  64.9    2.10            0.564809     5   Jan   
2     Hillsborough    Florida  59.0    3.64            0.540294     4   Jan   
3       Miami-Dade    Florida  65.7    2.28            0.412779     2   Jan   
4           Orange    Florida  59.1    2.66            0.408844     1   Jan   
...            ...        ...   ...     ...                 ...   ...   ...   
2395      Pinellas    Florida  63.7    3.62            0.766481     8   Dec   
2396        Fulton    Georgia  47.2    3.95            0.884067    10   Dec   
2397      Hennepin  Minnesota  32.1    2.31            0.437991     3   Dec   
2398      Hamilton       Ohio  41.2    2.09            0.696968     7   Dec   
2399       Natrona    Wyoming  30.3    0.14            0.682855     9   Dec   

      year  desirability_year  
0     2004         

In [219]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming final_results is your DataFrame already filtered for the year 2023
final_results = final_results[final_results['year'] == 2023]
# Create a new column for the legend
final_results['County_State'] = final_results['County_Name'] + ', ' + final_results['State']

# Create a line plot for temperature
fig_temp = px.line(
    final_results,
    x='month',
    y='temp',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Temperature in Top 10 Places to Retire (2023)'
)

# Update layout for temperature
fig_temp.update_layout(
    xaxis_title='Month',
    yaxis_title='Temperature (°F)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show temperature plot
fig_temp.show()

# Create a line plot for precipitation
fig_percip = px.line(
    final_results,
    x='month',
    y='percip',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Precipitation in Top 10 Places to Retire (2023)'
)

# Update layout for precipitation
fig_percip.update_layout(
    xaxis_title='Month',
    yaxis_title='Precipitation (mm)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show precipitation plot
fig_percip.show()

In [220]:
import pandas as pd
import plotly.express as px

# Assuming yearly_desirability is your DataFrame with yearly desirability data
# Make sure it has the columns 'County_Name', 'State', 'desirability_year'

# Create a new column for the legend
yearly_desirability['County_State'] = yearly_desirability['County_Name'] + ', ' + yearly_desirability['State']

# Sort values by desirability
yearly_desirability = yearly_desirability.sort_values(by='desirability_year', ascending=False)

# Create the horizontal bar plot
fig_bar = px.bar(
    yearly_desirability,
    x='desirability_year',
    y='County_State',
    color='County_State',  # Use the new column for color coding
    color_discrete_sequence=px.colors.qualitative.Set1,  # Adjust colors as needed
    title='Yearly Desirability in Top 10 Places to Retire (2023)',
    orientation='h'
)

# Update layout for better readability
fig_bar.update_layout(
    xaxis_title='Yearly Desirability Score',
    yaxis_title='County, State',
    legend_title='County, State',
)

# Show the bar plot
fig_bar.show()


In [221]:
# Create the figure
fig = px.line(monthly_averages, 
            x='month', 
            y='temp', 
            color='County_Name', 
            markers=True, 
            title='Monthly Temperature and Precipitation by County',
            labels={'temp': 'Temperature', 'month': 'Month'},
            line_shape='linear')

# Add precipitation as a second y-axis
fig.add_scatter(x=monthly_averages['month'], 
             y=monthly_averages['percip'], 
            mode='lines+markers', 
            name='Precipitation',
                 line=dict(dash='dash'))

# Update layout for dual axes
fig.update_layout(
    yaxis_title='Temperature',
    yaxis2=dict(title='Precipitation', overlaying='y', side='right'),
    xaxis_title='Month',
    legend_title='County Name',
)

# Show the plot
fig.show()

In [222]:

# Maybe contrast this wallethub (financially best ) with some other list of where people actually moved
#and look at the temperature differences between the 2 as welll as the "desired"
# Weight the scores based off of temp and precip 60/40
top10_wallet[['month','temp']].groupby('month').mean()

,temp
month,
Apr,63.9790
Aug,79.9610
Dec,51.2880
Feb,50.9125
Jan,48.3145
Jul,80.6195
Jun,77.7060
Mar,57.4745
May,71.0050


In [223]:
top10_wallet[['month','percip']].groupby('month').mean()

,percip
month,
Apr,2.83375
Aug,5.73500
Dec,2.25730
Feb,2.13320
Jan,2.10445
Jul,5.43905
Jun,5.52775
Mar,2.29705
May,3.61730


In [224]:
data = [
    {"Rank": 1, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 2, "County_Name": "Clark", "State": "Nevada"},
    {"Rank": 3, "County_Name": "Bexar", "State": "Texas"},
    {"Rank": 4, "County_Name": "Ada", "State": "Idaho"},
    {"Rank": 6, "County_Name": "New Hanover", "State": "North Carolina"},
    {"Rank": 7, "County_Name": "Collin", "State": "Texas"},
    {"Rank": 8, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 9, "County_Name": "Duval", "State": "Florida"},
    {"Rank": 10, "County_Name": "Wake", "State": "North Carolina"}
]
counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)


      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     1
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     1
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     1
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     1
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     1
...    ...    ...   ...   ...         ...      ...     ...   ...
2155  2019  48085   Dec  48.3      Collin    Texas    1.34     7
2156  2020  48085   Dec  47.3      Collin    Texas    3.09     7
2157  2021  48085   Dec  59.7      Collin    Texas    0.93     7
2158  2022  48085   Dec  48.4      Collin    Texas    2.53     7
2159  2023  48085   Dec  50.1      Collin    Texas    3.41     7

[2160 rows x 8 columns]


In [225]:
# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)

      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     1
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     1
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     1
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     1
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     1
...    ...    ...   ...   ...         ...      ...     ...   ...
2155  2019  48085   Dec  48.3      Collin    Texas    1.34     7
2156  2020  48085   Dec  47.3      Collin    Texas    3.09     7
2157  2021  48085   Dec  59.7      Collin    Texas    0.93     7
2158  2022  48085   Dec  48.4      Collin    Texas    2.53     7
2159  2023  48085   Dec  50.1      Collin    Texas    3.41     7

[2160 rows x 8 columns]


In [226]:
import pandas as pd

# Assuming top10_migration is your DataFrame containing the necessary columns

# Create a new column for year-month
top10_migration['year_month'] = top10_migration['year'].astype(str) + '-' + top10_migration['month']

# Calculate overall averages for each month across all years
overall_avg = top10_migration.groupby(['month', 'year']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
overall_avg.columns = ['month', 'year', 'overall_avg_temp', 'overall_avg_percip']

# Merge overall averages with the monthly data
monthly_averages = pd.merge(top10_migration, overall_avg, on=['month', 'year'])

# Calculate absolute differences from overall averages
monthly_averages['temp_diff'] = abs(monthly_averages['temp'] - monthly_averages['overall_avg_temp'])
monthly_averages['percip_diff'] = abs(monthly_averages['percip'] - monthly_averages['overall_avg_percip'])

# Weights for desirability components
w_T = 0.5  # weight for temperature
w_P = 0.2  # weight for precipitation
w_r = 0.3  # weight for rank

# Normalize the rank so that a lower rank yields a higher desirability contribution
monthly_averages['rank_adjusted'] = monthly_averages['Rank'].max() - monthly_averages['Rank'] + 1

# Define maximum possible differences for normalization
max_temp_diff = monthly_averages['temp_diff'].max()
max_percip_diff = monthly_averages['percip_diff'].max()

# Calculate desirability score
monthly_averages['desirability_month'] = (
    (1 - (monthly_averages['temp_diff'] / max_temp_diff)) * w_T +  # Smaller temp_diff is better
    (1 - (monthly_averages['percip_diff'] / max_percip_diff)) * w_P +  # Smaller percip_diff is better
    (1 - (monthly_averages['rank_adjusted'] / monthly_averages['rank_adjusted'].max())) * w_r  # Lower rank is better
)

# Calculate yearly desirability score by taking the mean of monthly scores per county and year
yearly_desirability = monthly_averages.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']

# Merge yearly desirability back into the monthly averages DataFrame
monthly_averages = pd.merge(monthly_averages, yearly_desirability, on=['County_Name', 'State', 'year'], how='left')

# Output the final results
final_results = monthly_averages[['County_Name', 'State', 'temp', 'percip', 'desirability_month', 'Rank', 'month', 'year', 'desirability_year']]
print(final_results)

      County_Name           State  temp  percip  desirability_month  Rank  \
0        Maricopa         Arizona  53.9    0.52            0.572690     1   
1           Duval         Florida  53.2    1.15            0.832341     9   
2      Miami-Dade         Florida  65.7    2.28            0.581629     8   
3             Ada           Idaho  28.7    1.83            0.458249     4   
4           Clark          Nevada  44.8    0.09            0.664816     2   
...           ...             ...   ...     ...                 ...   ...   
2155        Clark          Nevada  49.6    0.11            0.635413     2   
2156  New Hanover  North Carolina  50.0    8.31            0.751042     6   
2157         Wake  North Carolina  47.1    7.51            0.829264    10   
2158        Bexar           Texas  56.8    0.93            0.653417     3   
2159       Collin           Texas  50.1    3.41            0.832574     7   

     month  year  desirability_year  
0      Jan  2004           0.578157  

In [227]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming results_2023 is your DataFrame already filtered for the year 2023
results_2023 = results_2023[results_2023['year'] == 2023]
# Create a new column for the legend
results_2023['County_State'] = results_2023['County_Name'] + ', ' + results_2023['State']

# Create a line plot for temperature
fig_temp = px.line(
    results_2023,
    x='month',
    y='temp',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Temperature in Top 10 Places to Retire (2023)'
)

# Update layout for temperature
fig_temp.update_layout(
    xaxis_title='Month',
    yaxis_title='Temperature (°F)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show temperature plot
fig_temp.show()

# Create a line plot for precipitation
fig_percip = px.line(
    results_2023,
    x='month',
    y='percip',
    color='County_State',  # Use the new column for legend
    markers=True,
    title='Precipitation in Top 10 Places to Retire (2023)'
)

# Update layout for precipitation
fig_percip.update_layout(
    xaxis_title='Month',
    yaxis_title='Precipitation (mm)',  # Adjust unit as needed
    legend_title='County, State',
)

# Show precipitation plot
fig_percip.show()

In [228]:
import pandas as pd
import plotly.express as px

# Assuming yearly_desirability is your DataFrame with yearly desirability data
# Make sure it has the columns 'County_Name', 'State', 'desirability_year'

# Create a new column for the legend
yearly_desirability['County_State'] = yearly_desirability['County_Name'] + ', ' + yearly_desirability['State']

# Sort values by desirability
yearly_desirability = yearly_desirability.sort_values(by='desirability_year', ascending=False)

# Create the horizontal bar plot
fig_bar = px.bar(
    yearly_desirability,
    x='desirability_year',
    y='County_State',
    color='County_State',  # Use the new column for color coding
    color_discrete_sequence=px.colors.qualitative.Set1,  # Adjust colors as needed
    title='Yearly Desirability in Top 10 Places to Retire (2023) based on migration patterns',
    orientation='h'
)

# Update layout for better readability
fig_bar.update_layout(
    xaxis_title='Yearly Desirability Score',
    yaxis_title='County, State',
    legend_title='County, State',
)

# Show the bar plot
fig_bar.show()


In [236]:
import pandas as pd

# First list of counties
data1 = [
    {"Rank": 1, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 2, "County_Name": "Clark", "State": "Nevada"},
    {"Rank": 3, "County_Name": "Bexar", "State": "Texas"},
    {"Rank": 4, "County_Name": "Ada", "State": "Idaho"},
    {"Rank": 6, "County_Name": "New Hanover", "State": "North Carolina"},
    {"Rank": 7, "County_Name": "Collin", "State": "Texas"},
    {"Rank": 8, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 9, "County_Name": "Duval", "State": "Florida"},
    {"Rank": 10, "County_Name": "Wake", "State": "North Carolina"},
]

# Create DataFrames
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Combine both DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Group by County_Name and State, and calculate the average rank
final_df = combined_df.groupby(['County_Name', 'State']).agg({
    'Rank': 'mean'  # Average rank for counties appearing in both lists
}).reset_index()

# Sort by rank
final_df = final_df.sort_values(by='Rank')

# Display the final DataFrame
print(final_df)

top_20_list = pd.merge(clim, final_df, on=['County_Name', 'State'], how='inner')
print( top_20_list)


     County_Name           State  Rank
14        Orange         Florida   1.0
3          Clark          Nevada   2.0
8       Hennepin       Minnesota   3.0
1          Bexar           Texas   3.0
10      Maricopa         Arizona   3.5
9   Hillsborough         Florida   4.0
0            Ada           Idaho   4.0
11    Miami-Dade         Florida   5.0
2        Broward         Florida   5.0
13   New Hanover  North Carolina   6.0
7       Hamilton            Ohio   7.0
4         Collin           Texas   7.0
15      Pinellas         Florida   8.0
5          Duval         Florida   9.0
12       Natrona         Wyoming   9.0
6         Fulton         Georgia  10.0
16          Wake  North Carolina  10.0
      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52   3.5
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45   3.5
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00   3.5
3     2007   4013   Jan  49.8    Maric

In [265]:
# Define ideal conditions for each month using abbreviations
ideal_conditions = {
    'Jan': {'temp_range': (40, 60), 'percip_range': (1, 2)},
    'Feb': {'temp_range': (40, 65), 'percip_range': (1, 2)},
    'Mar': {'temp_range': (40, 70), 'percip_range': (1, 3)},
    'Apr': {'temp_range': (50, 75), 'percip_range': (2, 3)},
    'May': {'temp_range': (60, 80), 'percip_range': (1, 3)},
    'Jun': {'temp_range': (75, 85), 'percip_range': (1, 2)},
    'Jul': {'temp_range': (75, 90), 'percip_range': (0, 1)},
    'Aug': {'temp_range': (75, 90), 'percip_range': (1, 2)},
    'Sep': {'temp_range': (60, 80), 'percip_range': (2, 3)},
    'Oct': {'temp_range': (50, 75), 'percip_range': (1, 2)},
    'Nov': {'temp_range': (40, 70), 'percip_range': (1, 3)},
    'Dec': {'temp_range': (30, 60), 'percip_range': (1, 2)},
}



In [266]:
clim_2023 = clim[clim['year'] == 2023]


monthly_averages = clim_2023.groupby(['County_Name', 'State', 'month']).agg(
    avg_temp=('temp', 'mean'),
    avg_percip=('percip', 'mean')
).reset_index()


monthly_averages.head()


,County_Name,State,month,avg_temp,avg_percip
0,Abbeville,South Carolina,Apr,61.1,6.18
1,Abbeville,South Carolina,Aug,80.0,5.13
2,Abbeville,South Carolina,Dec,47.8,4.09
3,Abbeville,South Carolina,Feb,54.8,4.27
4,Abbeville,South Carolina,Jan,48.6,8.26


In [267]:

def evaluate_conditions(row):
    month_name = row['month']
    temp_range = ideal_conditions[month_name]['temp_range']
    percip_range = ideal_conditions[month_name]['percip_range']
    
    # Score temperature
    temp_score = 0
    if temp_range[0] <= row['avg_temp'] <= temp_range[1]:
        temp_score = 1  # Perfect score for being within range
    else:
        # Scale score based on distance from ideal range
        temp_score = max(0, min(1, (temp_range[1] - row['avg_temp']) / (temp_range[1] - temp_range[0])))

    # Score precipitation
    percip_score = 0
    if percip_range[0] <= row['avg_percip'] <= percip_range[1]:
        percip_score = 1  # Perfect score for being within range
    else:
        # Scale score based on distance from ideal range
        percip_score = max(0, min(1, (percip_range[1] - row['avg_percip']) / (percip_range[1] - percip_range[0])))
    
    return (temp_score *.6  + percip_score) *.4 / 2  # Average score

# Apply evaluation function
monthly_averages['score'] = monthly_averages.apply(evaluate_conditions, axis=1)


In [268]:
# Apply evaluation function
final_scores = monthly_averages.groupby(['County_Name', 'State']).agg({'score': 'mean'}).reset_index()

# Sort by score to find the most desirable county
best_weather_locations = final_scores.sort_values(by='score', ascending = False)

# Display the most desirable county
print("Most Desirable County in 2023:")
print(best_weather_locations.head(10))


Most Desirable County in 2023:
     County_Name       State  score
1691  Los Alamos  New Mexico   0.32
866        Emery        Utah   0.32
668        Crook      Oregon   0.32
1252     Hidalgo  New Mexico   0.32
635     Costilla    Colorado   0.32
1910    Minidoka       Idaho   0.32
1196      Harney      Oregon   0.32
2725        Taos  New Mexico   0.32
2496    Santa Fe  New Mexico   0.32
91        Asotin  Washington   0.32


In [269]:
desired_counties = [
    {'County_Name': 'Los Alamos', 'State': 'New Mexico'},
    {'County_Name': 'Emery', 'State': 'Utah'},
    {'County_Name': 'Crook', 'State': 'Oregon'},
    {'County_Name': 'Hidalgo', 'State': 'New Mexico'},
    {'County_Name': 'Costilla', 'State': 'Colorado'},
    {'County_Name': 'Minidoka', 'State': 'Idaho'},
    {'County_Name': 'Harney', 'State': 'Oregon'},
    {'County_Name': 'Taos', 'State': 'New Mexico'},
    {'County_Name': 'Santa Fe', 'State': 'New Mexico'},
    {'County_Name': 'Asotin', 'State': 'Washington'}
]

# Convert list of dicts to DataFrame for filtering
desired_df = pd.DataFrame(desired_counties)

# Merge to find matches in climate data
filtered_climate_data = pd.merge(desired_df, clim_2023, on=['County_Name', 'State'], how='left')


filtered_climate_data['County_State'] = filtered_climate_data['County_Name'] + ', ' + filtered_climate_data['State']

    # Create a line plot for temperature
fig_temp = px.line(
        filtered_climate_data,
        x='month',
        y='temp',
        color='County_State',  # Use the new column for legend
        markers=True,
        title='Temperature in Desired Counties'
    )

    # Update layout for temperature
fig_temp.update_layout(
        xaxis_title='Month',
        yaxis_title='Temperature (°F)',
        legend_title='County, State',
    )

    # Show temperature plot
fig_temp.show()

    # Create a line plot for precipitation
fig_percip = px.line(
        filtered_climate_data,
        x='month',
        y='percip',
        color='County_State',  # Use the new column for legend
        markers=True,
        title='Precipitation in Counties with best weather conditions'
    )

    # Update layout for precipitation
fig_percip.update_layout(
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        legend_title='County, State',
    )

    # Show precipitation plot
fig_percip.show()